In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE=160
BATCH_SIZE=32
EPOCHS=20

In [ ]:
#API for loading images from directory
data=tf.keras.preprocessing.image_dataset_from_directory(
    "trainingCarD",
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

In [ ]:
class_names=data.class_names
class_names

In [ ]:
len(data)             

In [ ]:
plt.figure(figsize=(10,10))
for image_batch,label_batch in data.take(1):
    print(image_batch.shape)
    print(label_batch.numpy())
    for i in range(10):
        img=plt.subplot(2,5,i+1)
        plt.imshow(image_batch[0].numpy().astype('uint8'))
        plt.title(class_names[label_batch[i]])
        plt.axis("off")

In [ ]:
#Train and test dataset split (manually)
def test_train_split(data,train_split,test_split,shuffle,shuffle_size):
    data_size=len(data)
    if shuffle:
        data=data.shuffle(shuffle_size,seed=12)
    train_size=int(train_split*data_size)
    test_size=int(test_split*data_size)
    
    train_data=data.take(train_size)
    test_data=data.skip(train_size)
    
    return train_data,test_data

In [ ]:
train_data,test_data=test_train_split(data=data,train_split=0.8,test_split=0.2,shuffle=True,shuffle_size=10000)

In [ ]:
len(train_data)

In [ ]:
len(test_data)

In [ ]:
#For performance improvement
train_data=train_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_data=test_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
#Now we will scale the images with data augmentation 
resize_and_rescale=tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [ ]:
data_augmentation=tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)                                   
])

In [ ]:
n_classes=2
model=models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,3)),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(n_classes,activation='softmax')
])
model.build(input_shape=(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,3))
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'],
)

In [ ]:
history=model.fit(
    train_data,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=test_data
)

In [ ]:
plt.plot(history.history['accuracy'], label='train') 
plt.plot(history.history['val_accuracy'], label='test') 
plt.legend()
plt.show()

In [ ]:
print('Test accuracy achieved', history.history['val_accuracy'][-2])